In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.utils.data as data
from torch.utils.tensorboard import SummaryWriter
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import time, os, copy, argparse
import multiprocessing
from torchsummary import summary
from matplotlib import pyplot as plt

In [ ]:
train_directory = '/content/drive/MyDrive/TOMATO_LEAF_DATASET/train'
valid_directory = '/content/drive/MyDrive/TOMATO_LEAF_DATASET/val'

In [ ]:
PATH="/content/drive/MyDrive/TOMATO_LEAF_DATASET/weights/MobileNet_bs256_5epoch.pth"
bs = 512
num_epochs = 5
num_classes = 2
num_cpu = multiprocessing.cpu_count()

image_transforms = { 
    'train': transforms.Compose([
        transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
    ])
}
 
dataset = {
    'train': datasets.ImageFolder(root=train_directory, transform=image_transforms['train']),
    'valid': datasets.ImageFolder(root=valid_directory, transform=image_transforms['valid'])
}
 
dataset_sizes = {
    'train':len(dataset['train']),
    'valid':len(dataset['valid'])
}


In [ ]:
dataset_sizes

{'train': 16361, 'valid': 824}

In [ ]:
dataset

{'train': Dataset ImageFolder
     Number of datapoints: 16361
     Root location: /content/drive/MyDrive/TOMATO_LEAF_DATASET/train
     StandardTransform
 Transform: Compose(
                RandomResizedCrop(size=(256, 256), scale=(0.8, 1.0), ratio=(0.75, 1.3333), interpolation=bilinear)
                RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
                RandomHorizontalFlip(p=0.5)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ), 'valid': Dataset ImageFolder
     Number of datapoints: 824
     Root location: /content/drive/MyDrive/TOMATO_LEAF_DATASET/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
                CenterCrop(size=(224, 224))
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.2

In [ ]:
dataloaders = {
    'train':data.DataLoader(dataset['train'], batch_size=bs, shuffle=True,
                            num_workers=num_cpu, pin_memory=True, drop_last=True),
    'valid':data.DataLoader(dataset['valid'], batch_size=bs, shuffle=True,
                            num_workers=num_cpu, pin_memory=True, drop_last=True)
}


class_names = dataset['train'].classes
print("Classes:", class_names)

Classes: ['infected', 'uninfected']


In [ ]:
print("Training-set size:",dataset_sizes['train'],
      "\nValidation-set size:", dataset_sizes['valid'])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Training-set size: 16361 
Validation-set size: 824


In [ ]:
print("\nLoading mobilenetv2 as feature extractor ...\n")
model_ft = models.mobilenet_v2(pretrained=True)    

for params in list(model_ft.parameters())[0:-5]:
    params.requires_grad = False

num_ftrs=model_ft.classifier[-1].in_features
model_ft.classifier=nn.Sequential(nn.Dropout(p=0.2, inplace=False),
    nn.Linear(in_features=num_ftrs, out_features=num_classes, bias=True)
    )    


Loading mobilenetv2 as feature extractor ...



Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

In [ ]:
model_ft = model_ft.to(device)

In [ ]:
print('Model Summary:-\n')
for num, (name, param) in enumerate(model_ft.named_parameters()):
    print(num, name, param.requires_grad )
summary(model_ft, input_size=(3, 224, 224))
print(model_ft)

Model Summary:-

0 features.0.0.weight False
1 features.0.1.weight False
2 features.0.1.bias False
3 features.1.conv.0.0.weight False
4 features.1.conv.0.1.weight False
5 features.1.conv.0.1.bias False
6 features.1.conv.1.weight False
7 features.1.conv.2.weight False
8 features.1.conv.2.bias False
9 features.2.conv.0.0.weight False
10 features.2.conv.0.1.weight False
11 features.2.conv.0.1.bias False
12 features.2.conv.1.0.weight False
13 features.2.conv.1.1.weight False
14 features.2.conv.1.1.bias False
15 features.2.conv.2.weight False
16 features.2.conv.3.weight False
17 features.2.conv.3.bias False
18 features.3.conv.0.0.weight False
19 features.3.conv.0.1.weight False
20 features.3.conv.0.1.bias False
21 features.3.conv.1.0.weight False
22 features.3.conv.1.1.weight False
23 features.3.conv.1.1.bias False
24 features.3.conv.2.weight False
25 features.3.conv.3.weight False
26 features.3.conv.3.bias False
27 features.4.conv.0.0.weight False
28 features.4.conv.0.1.weight False
29 fea

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=30):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    writer = SummaryWriter()
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device, non_blocking=True)
                labels = labels.to(device, non_blocking=True)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            if phase == 'train':
                train_losses=[]
                train_acc=[]
                writer.add_scalar('Train/Loss', epoch_loss, epoch)
                writer.add_scalar('Train/Accuracy', epoch_acc, epoch)
                train_losses.append(epoch_loss)
                train_acc.append(epoch_acc)
                writer.flush()
            else:
                val_losses=[]
                val_acc=[]
                writer.add_scalar('Valid/Loss', epoch_loss, epoch)
                writer.add_scalar('Valid/Accuracy', epoch_acc, epoch)
                val_losses.append(epoch_loss)
                val_acc.append(epoch_acc)
                writer.flush()

            # deep copy the model
            if phase == 'valid' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,num_epochs=num_epochs)
torch.save(model_ft, PATH)

Epoch 0/4
----------
train Loss: 0.3540 Acc: 0.8192
valid Loss: 0.2895 Acc: 0.4381

Epoch 1/4
----------
train Loss: 0.1848 Acc: 0.9045
valid Loss: 0.1924 Acc: 0.5522

Epoch 2/4
----------
train Loss: 0.1307 Acc: 0.9342
valid Loss: 0.1465 Acc: 0.5862

Epoch 3/4
----------
train Loss: 0.1083 Acc: 0.9423
valid Loss: 0.1246 Acc: 0.5922

Epoch 4/4
----------
train Loss: 0.0940 Acc: 0.9468
valid Loss: 0.1156 Acc: 0.5934

Training complete in 110m 13s
Best val Acc: 0.593447


In [ ]:
model1=torch.load("/content/drive/MyDrive/Dataset/models/mobilenet_v2_tomato.pth") 
model1.eval()

In [ ]:
import PIL.Image as Image
transform = transforms.Compose([ transforms.Resize(size=256),transforms.CenterCrop(size=224),transforms.ToTensor(),transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])])
img = Image.open('/content/drive/MyDrive/Dataset/test2.jpeg')  
x = transform(img) 
x = x.unsqueeze(0)  

output = model1(x) 
print(output)
pred = torch.argmax(output, 1) 
print('Image predicted as ', pred)

tensor([[-0.4199,  0.3027]], grad_fn=<AddmmBackward0>)
Image predicted as  tensor([1])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

cm = metrics.confusion_matrix(test_batch.classes, y_pred)

plt.imshow(cm, cmap=plt.cm.Blues)
plt.xlabel("Predicted labels")
plt.ylabel("True labels")
plt.xticks([], [])
plt.yticks([], [])
plt.title('Confusion matrix ')
plt.colorbar()
plt.show()